# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных


children — количество детей в семье

days_employed — общий трудовой стаж в днях

dob_years — возраст клиента в годах

education — уровень образования клиента

education_id — идентификатор уровня образования

family_status — семейное положение

family_status_id — идентификатор семейного положения

gender — пол клиента

income_type — тип занятости

debt — имел ли задолженность по возврату кредитов

total_income — ежемесячный доход

purpose — цель получения кредита

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

In [2]:
bank_clients = pd.read_csv('/datasets/data.csv')

До начала всей работы проверим дубликаты и удалим

In [3]:
x = bank_clients.duplicated().sum()
print('На данный момент дубликатов', x)
bank_clients = bank_clients.drop_duplicates().reset_index(drop=True)


На данный момент дубликатов 54


Изучим таблицу
Изучим информацию по таблице


In [4]:
print(bank_clients.info())
print('')
print ('Пропущено в столбцах "days_employed" и "total_income"', 21525-19351)
print('')
bank_clients.head(11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
children            21471 non-null int64
days_employed       19351 non-null float64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        19351 non-null float64
purpose             21471 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None

Пропущено в столбцах "days_employed" и "total_income" 2174



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

21,5 тычяча строк!!! Вручную просмотреть такую таблицу будет сложновато. 
1. В столбцах days_employed и total_income есть нулевые значения
2. Почему-то days_employed - число с плавающей точкой
3. family_status и family_status_id, education и education_id нужно посмотреть в чем принципиальная разница и есть ли необходимость в этих парах, либо они дублирующиеся
4. gender - стоит посмотреть уникальные значение. Больше 2х - будет чутка странно, в Беларуси не поймут(хотя этот пункт под вопросом)
5. Прверить total_income, у всех ли он положительный. Если нет, сразу можно отправлять домой. Хотя, если будут люди с нулевым или отрицательным
6. total_income и days_employed с одинаковым количеством пропусков. Совпадение? Не думаю!

Задача поставленная нам -  узнать "влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок"
Поэтому есть шанс что total_income нам не пригодится)))




Но все дело в том, что у нас нет ни имен, ни фамилий ни даже ID людей, поэтому при удалении дубликатов мы можем удалить часть необходимых данных:

### Шаг 2. Предобработка данных

### Обработка пропусков

Начнемс. Точнее проолжим, ведь я уже это прошел и не сохранил, просто у кого-то не оттуда руки, поэтому буду переделывать! 
Смотрим days_employed

In [5]:
print(bank_clients[bank_clients['days_employed'].isnull()].count()) 

children            2120
days_employed          0
dob_years           2120
education           2120
education_id        2120
family_status       2120
family_status_id    2120
gender              2120
income_type         2120
debt                2120
total_income           0
purpose             2120
dtype: int64


Получается, что во всех пропущенных строках рабочих дней пропущены и заработки
Из этого следует вывод, что это неслучайные пропуски. Есть вероятность что они работают, но "вчерную"
Далее я проверил вероятность того, что все 2174 человека являются 18-летними и не могут иметь стаж

In [6]:
unemployed = bank_clients[bank_clients['days_employed'].isnull()]
print(unemployed.groupby('dob_years')['dob_years'].count())

dob_years
0     10
19     1
20     5
21    18
22    17
23    35
24    21
25    23
26    35
27    36
28    57
29    49
30    56
31    64
32    36
33    51
34    67
35    63
36    62
37    52
38    53
39    50
40    64
41    58
42    64
43    49
44    42
45    50
46    46
47    56
48    45
49    50
50    50
51    50
52    53
53    44
54    52
55    48
56    51
57    52
58    51
59    33
60    36
61    37
62    35
63    29
64    34
65    20
66    19
67    16
68     9
69     5
70     3
71     5
72     2
73     1
Name: dob_years, dtype: int64


10 человек из 21,5к младше 18 лет - меньше 0,05%. Расслабились и отпустили.
Хотя есть шанс, что на общей выборке их будет больше
Все ок. данные идентичны. Проверяем детей

In [7]:
print(bank_clients['children'].value_counts())

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


"-1" ребенок настораживает. на мой взгляд тут возможны 3 варианта: 
 - Ошибка при заполнении (тогда просто переводим в 1)
 - "смерть", как бы он был, а теперь нет (маловероятно)
 - Беременность (нужно проверить. Если да, тоже переводить в 1)
 
Теоретически можно сразу перевести, но интересно найти закономерность 


In [8]:
print(bank_clients[bank_clients['children'] == -1])

       children  days_employed  dob_years            education  education_id  \
291          -1   -4417.703588         46              среднее             1   
705          -1    -902.084528         50              среднее             1   
742          -1   -3174.456205         57              среднее             1   
800          -1  349987.852217         54              среднее             1   
941          -1            NaN         57              Среднее             1   
1363         -1   -1195.264956         55              СРЕДНЕЕ             1   
1929         -1   -1461.303336         38              среднее             1   
2073         -1   -2539.761232         42              среднее             1   
3813         -1   -3045.290443         26              Среднее             1   
4199         -1    -901.101738         41              среднее             1   
4400         -1  398001.302888         64              СРЕДНЕЕ             1   
4540         -1   -1811.899756         3

Есть как М, так и Ж, значит это не беременные. Отправляем всех -1 детей к однодетным

In [9]:
bank_clients.loc[bank_clients['children'] == -1, 'children'] = 1
print(bank_clients['children'].value_counts())


0     14107
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64


Вот теперь это похоже на правду. А что за 76 человек с 20 детьми?

In [10]:
children_20 = bank_clients[bank_clients['children'] == 20]
print(children_20.head(15))
print('')
print('')
print(bank_clients.info())

      children  days_employed  dob_years education  education_id  \
606         20    -880.221113         21   среднее             1   
720         20    -855.595512         44   среднее             1   
1074        20   -3310.411598         56   среднее             1   
2510        20   -2714.161249         59    высшее             0   
2940        20   -2161.591519          0   среднее             1   
3301        20            NaN         35   среднее             1   
3395        20            NaN         56    высшее             0   
3670        20    -913.161503         23   среднее             1   
3696        20   -2907.910616         40   среднее             1   
3734        20    -805.044438         26    высшее             0   
3876        20   -8190.644409         45   среднее             1   
5017        20    -231.783475         37   среднее             1   
5312        20   -2047.754733         24   среднее             1   
5346        20  361744.836360         64   средн

Теоретически, если бы все они брали кредиты на недвижимость и были старше 40 лет (это фантазия), можно было бы предположить,
Что случилось невероятное и они многодетные от слова совсем. 


In [11]:
print(children_20.groupby('dob_years')['dob_years'].count())


dob_years
0     1
21    1
23    1
24    1
25    1
26    1
27    2
29    2
30    3
31    2
32    2
33    2
34    3
35    2
36    2
37    4
38    1
39    1
40    4
41    2
42    3
43    2
44    2
45    3
46    3
48    1
49    3
50    3
51    1
52    1
53    1
54    1
55    1
56    5
57    1
59    2
60    1
61    1
62    1
64    1
69    1
Name: dob_years, dtype: int64


ну или 76 человек - только мужчины, которые жизнь положили на увеличение потомков

In [12]:
print(children_20.groupby('gender')['gender'].count())

gender
F    47
M    29
Name: gender, dtype: int64


Ни то ни другое истиной не является и т.к. в выборке с увеличением детей количество людей снижается, смею предположить, 
что "20" является ошибкой, если не преднамеренным исправлением
Закидываем всех с 20 детьми к 2м детям, только нужно проверить, не сильно ли изменит это выборку с долгами

In [13]:
print(bank_clients.groupby('children')['debt'].value_counts())

children  debt
0         0       13044
          1        1063
1         0        4411
          1         445
2         0        1858
          1         194
3         0         303
          1          27
4         0          37
          1           4
5         0           9
20        0          68
          1           8
Name: debt, dtype: int64


Повальных долгов нет, и 76 человек из всей выборки - меньше процента. Отправляем их к двудетным

In [14]:
bank_clients.loc[bank_clients['children'] == 20, 'children'] = 2

print(bank_clients['children'].value_counts())


0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


Найдем все уникальные "профессии"

In [15]:
print(bank_clients['income_type'].unique())


['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Тут без повторов, и на том спасибо. Давайте узнаем сколько кто не получает зарплату по нашим данным

In [16]:
unemployed_categories = bank_clients[bank_clients['total_income'].isnull()]
print(unemployed_categories['income_type'].unique())

['пенсионер' 'госслужащий' 'компаньон' 'сотрудник' 'предприниматель']


Типов всего 5, глянем, кого больше

In [17]:

print(unemployed_categories.groupby('income_type')['income_type'].count())

income_type
госслужащий         145
компаньон           503
пенсионер           394
предприниматель       1
сотрудник          1077
Name: income_type, dtype: int64


Начнем избавляться от пропусков с самой многочисленной группы - "сотрудники"
Посчитаем средний заработок всех "сотрудников", что есть у нас в таблице и заполним пропуски медианой
Учтем их

In [18]:
employe_m = bank_clients[(bank_clients['income_type'] == 'сотрудник') 
                         & (bank_clients['gender'] == 'M')]['total_income'].dropna().median()

print('Медиана для "сотрудник мужчина" {:.1f}'.format(employe_m))

employe_f = bank_clients[(bank_clients['income_type'] == 'сотрудник') 
                         & (bank_clients['gender'] == 'F')]['total_income'].dropna().median()

print('Медиана для "сотрудник женщина" {:.1f}'.format(employe_f))


Медиана для "сотрудник мужчина" 162161.2
Медиана для "сотрудник женщина" 130615.6


По такой же аналогии просчитаем все остальные категории


In [19]:

civil_servant_m = bank_clients[(bank_clients['income_type'] == 'госслужащий') 
                         & (bank_clients['gender'] == 'M')]['total_income'].dropna().median()
print('Медиана для "госслужащий мужчина" {:.1f}'.format(civil_servant_m))
civil_servant_f = bank_clients[(bank_clients['income_type'] == 'госслужащий') 
                         & (bank_clients['gender'] == 'F')]['total_income'].dropna().median()
print('Медиана для "госслужащий женщина" {:.1f}'.format(civil_servant_f))

companion_m = bank_clients[(bank_clients['income_type'] == 'компаньон') 
                         & (bank_clients['gender'] == 'M')]['total_income'].dropna().median()
print('Медиана для "компаньон мужчина" {:.1f}'.format(companion_m))
companion_f = bank_clients[(bank_clients['income_type'] == 'компаньон') 
                         & (bank_clients['gender'] == 'F')]['total_income'].dropna().median()
print('Медиана для "компаньон женщина" {:.1f}'.format(companion_f))


rest_m = bank_clients[(bank_clients['income_type'] == 'пенсионер') 
                         & (bank_clients['gender'] == 'M')]['total_income'].dropna().median()
print('Медиана для "пенсионер мужчина" {:.1f}'.format(rest_m))
rest_f = bank_clients[(bank_clients['income_type'] == 'пенсионер') 
                         & (bank_clients['gender'] == 'F')]['total_income'].dropna().median()
print('Медиана для "пенсионер женщина" {:.1f}'.format(rest_f))

businessman = bank_clients[(bank_clients['income_type'] == 'предприниматель')]['total_income'].dropna().median()

print('Медиана для "предприниматель" {:.1f}'.format(businessman))



Медиана для "госслужащий мужчина" 185964.9
Медиана для "госслужащий женщина" 136982.5
Медиана для "компаньон мужчина" 196818.8
Медиана для "компаньон женщина" 160820.8
Медиана для "пенсионер мужчина" 130739.8
Медиана для "пенсионер женщина" 115807.8
Медиана для "предприниматель" 499163.1


Мы узнали данные для заполнения, теперь будем заполнять пропуски
Сначала заменим все пропуски на 0, а после в зависимости от пола и занятости заполним все 0

In [20]:
bank_clients['total_income'] = bank_clients['total_income'].fillna(0)
bank_clients.loc[:, 'total_income'] = bank_clients.loc[:, 'total_income'].astype('int')

In [21]:
x = bank_clients[bank_clients['total_income'] == 0].count()
x

children            2120
days_employed          0
dob_years           2120
education           2120
education_id        2120
family_status       2120
family_status_id    2120
gender              2120
income_type         2120
debt                2120
total_income        2120
purpose             2120
dtype: int64

In [22]:


def zero_incom_delete(row):
    income_type = row['income_type'] 
    gender = row['gender'] 
    total_income = row['total_income'] 
    if total_income == 0:
        if income_type == 'пенсионер':
            if gender == 'm':
                return rest_m
            else:
                return rest_f
        if income_type == 'госслужащий':
            if gender == 'm':
                return civil_servant_m
            else:
                return civil_servant_f
        if income_type == 'компаньон':
            if gender == 'm':
                return companion_m
            else:
                return companion_f
        if income_type == 'сотрудник':
            if gender == 'm':
                return employe_m
            else:
                return employe_f
        if income_type == 'предприниматель':
                return businessman
    return total_income

bank_clients['total_income'] = bank_clients.apply(zero_incom_delete, axis=1)

bank_clients.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.000000,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.000000,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.000000,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.000000,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.000000,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.000000,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.000000,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.000000,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.000000,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.000000,покупка жилья для семьи


АЕ! Кто молодец? Я!
Заполнили total_income, притом не какой-то общей медианой, а медианой по полу и занятости!

Что с days_employed? зачем там плавающая точка, это же дни
Никто не запоминает с количество отработаннх дней с точностью до месяца, за малым исключением, 
Поэтому смело можно избавляться от данных после запятой. 

In [23]:
bank_clients['days_employed'] = bank_clients['days_employed'].fillna(0)
print(bank_clients[bank_clients['days_employed'] < 1].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18026 entries, 0 to 21470
Data columns (total 12 columns):
children            18026 non-null int64
days_employed       18026 non-null float64
dob_years           18026 non-null int64
education           18026 non-null object
education_id        18026 non-null int64
family_status       18026 non-null object
family_status_id    18026 non-null int64
gender              18026 non-null object
income_type         18026 non-null object
debt                18026 non-null int64
total_income        18026 non-null float64
purpose             18026 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.8+ MB
None


18к неположительных часов рабочих. У нас спрашивают зависимость от иной переменной, но "странных" данных слишком много, 
Поэтому предположим, что знак минус, как и с детьми это дефис

In [24]:
bank_clients.loc[bank_clients.loc[:, 'days_employed'] < 0, 
         'days_employed'] = -bank_clients.loc[bank_clients.loc[:, 'days_employed'] < 0, 'days_employed']

print(bank_clients['days_employed'].value_counts())


0.000000       2120
1645.463049       1
6620.396473       1
1238.560080       1
3047.519891       1
               ... 
2849.351119       1
5619.328204       1
448.829898        1
1687.038672       1
206.107342        1
Name: days_employed, Length: 19352, dtype: int64


Находим людей, у которых стаж длиннее жизни)))

In [25]:
days_before_work = 18*365.25
bank_clients['check'] = bank_clients['dob_years']*365 - days_before_work - bank_clients['days_employed']
bank_clients.loc[:, 'check'] = bank_clients.loc[:, 'check'].astype('int')

print(bank_clients[bank_clients['check'] < 0].groupby('debt')['debt'].count())

debt
0    3725
1     218
Name: debt, dtype: int64


Подавляющее большинство не имеют долгов, а это значит что увеличение стажа не яалялось преднамеренным для согласования кредита
Давайте попробуем заполнить пропущенные данные с помощью медиан


In [26]:
days_employed_employe_m = bank_clients[(bank_clients['income_type'] == 'сотрудник') 
                         & (bank_clients['gender'] == 'M') & (bank_clients['days_employed'] != 0)]['days_employed'].median()
print('Медиана для "сотрудник мужчина" {:.1f}'.format(days_employed_employe_m))                                         

days_employed_employe_f = bank_clients[(bank_clients['income_type'] == 'сотрудник') 
                         & (bank_clients['gender'] == 'F') & (bank_clients['days_employed'] != 0)]['days_employed'].median()

print('Медиана для "сотрудник женщина" {:.1f}'.format(days_employed_employe_f))

days_employed_civil_servant_m = bank_clients[(bank_clients['income_type'] == 'госслужащий') 
                         & (bank_clients['gender'] == 'M') & (bank_clients['days_employed'] != 0)]['days_employed'].median()
print('Медиана для "госслужащий мужчина" {:.1f}'.format(days_employed_civil_servant_m))
days_employed_civil_servant_f = bank_clients[(bank_clients['income_type'] == 'госслужащий') 
                         & (bank_clients['gender'] == 'F') & (bank_clients['days_employed'] != 0)]['days_employed'].median()
print('Медиана для "госслужащий женщина" {:.1f}'.format(days_employed_civil_servant_f))

days_employed_companion_m = bank_clients[(bank_clients['income_type'] == 'компаньон') 
                         & (bank_clients['gender'] == 'M') & (bank_clients['days_employed'] != 0)]['days_employed'].median()
print('Медиана для "компаньон мужчина" {:.1f}'.format(days_employed_companion_m))
days_employed_companion_f = bank_clients[(bank_clients['income_type'] == 'компаньон') 
                         & (bank_clients['gender'] == 'F') & (bank_clients['days_employed'] != 0)]['days_employed'].median()
print('Медиана для "компаньон женщина" {:.1f}'.format(days_employed_companion_f))


days_employed_rest_m = bank_clients[(bank_clients['income_type'] == 'пенсионер') 
                         & (bank_clients['gender'] == 'M') & (bank_clients['days_employed'] != 0)]['days_employed'].median()
print('Медиана для "пенсионер мужчина" {:.1f}'.format(days_employed_rest_m))
days_employed_rest_f = bank_clients[(bank_clients['income_type'] == 'пенсионер') 
                         & (bank_clients['gender'] == 'F') & (bank_clients['days_employed'] != 0)]['days_employed'].median()
print('Медиана для "пенсионер женщина" {:.1f}'.format(days_employed_rest_f))

days_employed_businessman = bank_clients[(bank_clients['income_type'] == 'предприниматель') & (bank_clients['days_employed'] != 0)]['days_employed'].median()

print('Медиана для "предприниматель" {:.1f}'.format(days_employed_businessman))

Медиана для "сотрудник мужчина" 1362.7
Медиана для "сотрудник женщина" 1718.5
Медиана для "госслужащий мужчина" 2659.3
Медиана для "госслужащий женщина" 2705.8
Медиана для "компаньон мужчина" 1472.9
Медиана для "компаньон женщина" 1584.3
Медиана для "пенсионер мужчина" 361781.5
Медиана для "пенсионер женщина" 366182.9
Медиана для "предприниматель" 520.8


Нашли медианы рабочих дней. Пора подставлять

In [27]:
bank_clients.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,check
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.000000,покупка жилья,317
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.000000,приобретение автомобиля,2540
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.000000,покупка жилья,-152
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.000000,дополнительное образование,980
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.000000,сыграть свадьбу,-327495
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.000000,покупка жилья,2354
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.000000,операции с жильем,6241
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.000000,образование,11522
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.000000,на проведение свадьбы,-729
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.000000,покупка жилья для семьи,6201


In [28]:
def zero_days_employed_delete(row):
    income_type = row['income_type'] 
    gender = row['gender'] 
    days_employed = row['days_employed'] 
    if days_employed == 0:
        if income_type == 'пенсионер':
            if gender == 'm':
                return days_employed_rest_m
            else:
                return days_employed_rest_f
        if income_type == 'госслужащий':
            if gender == 'm':
                return days_employed_civil_servant_m
            else:
                return days_employed_civil_servant_f
        if income_type == 'компаньон':
            if gender == 'm':
                return days_employed_companion_m
            else:
                return days_employed_companion_f
        if income_type == 'сотрудник':
            if gender == 'm':
                return days_employed_employe_m
            else:
                return days_employed_employe_f
        if income_type == 'предприниматель':
            return days_employed_businessman
    return days_employed

bank_clients['days_employed'] = bank_clients.apply(zero_days_employed_delete, axis=1)

print(bank_clients['days_employed'].value_counts())

1718.507316      1078
1584.335109       503
366182.933484     394
2705.835929       145
520.848083          2
                 ... 
2849.351119         1
5619.328204         1
448.829898          1
1687.038672         1
582.538413          1
Name: days_employed, Length: 19354, dtype: int64


Вот и славненько. Людей без стажа или с отрицательным стажем у нас нет!

### Вывод

Данные очень гразные
Есть вероятность, и очень большая, что в данных очень много пасхалок, половину из которых я не нашел
Заполнил пустые ячейки медианами соотетствующих категорий.
Вызывает огромный вопрос 4000 человек, стаж которых больше их взрослой жизни, но есть вероятность, что люди трудятся на 2х и более работах
А если и нет, то убрать "этих людей" мы не можем - они составляют почти 20%
Можно было бы по медиане присвоить и им стаж, но думаю, это решение будет неверным

### Замена типа данных

Убираем знаки после запятой в days_employed и total_income


In [29]:
bank_clients.loc[:, 'days_employed'] = bank_clients.loc[:, 'days_employed'].astype('int')
bank_clients.loc[:, 'total_income'] = bank_clients.loc[:, 'total_income'].astype('int')
bank_clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,check
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,317
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2540
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,-152
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,980
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,-327495


In [30]:
x = bank_clients.duplicated().sum()
print('На данный момент дубликатов', x)

На данный момент дубликатов 0


Вывод: После удаления точек после запятой данные не пострадади ни на 1 индекс

### Вывод

Замена типа данных появляется периодически за все прохождение задания. Это связано с некорректным подбором типа данных в начальной таблице и последующим его изменением

При всем желании и уважении, исправить пропуски без замены 

### Обработка дубликатов

In [31]:
print(bank_clients['education'].value_counts())

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


Обнаружилась кривизна заполнения. исправляем

In [32]:
bank_clients['education'] = bank_clients['education'].str.lower()
print(bank_clients['education'].value_counts())


среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


Проверяем соответствие столбцов education и education_id

In [33]:
print(bank_clients.info())
print('')
print(bank_clients.groupby('education')['education_id'].value_counts())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 13 columns):
children            21471 non-null int64
days_employed       21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null int64
purpose             21471 non-null object
check               21471 non-null int64
dtypes: int64(8), object(5)
memory usage: 2.1+ MB
None

education            education_id
высшее               0                5251
начальное            3                 282
неоконченное высшее  2                 744
среднее              1               15188
ученая степень       4                   6
Name: education_id, dtype: int64


Все ок. данные идентичны
Так же проверим и family_status и family_status_id


In [34]:
bank_clients['family_status'] = bank_clients['family_status'].str.lower()
print(bank_clients.groupby('family_status')['family_status_id'].value_counts())

family_status          family_status_id
в разводе              3                    1195
вдовец / вдова         2                     959
гражданский брак       1                    4163
женат / замужем        0                   12344
не женат / не замужем  4                    2810
Name: family_status_id, dtype: int64


Тут все ровно. только вот для банка нет разницы между "Не женат / не замужем", 
"в разводе" и "вдовец / вдова"
Я бы еще туда и гражданский брак отправил, т.к. по законодательству, по крайней мере РБ, 
Нет разницы, но, на всякий случай, оставим

In [35]:
bank_clients.loc[bank_clients.loc[:, 'family_status_id'] > 1, 'family_status_id'] = 4
bank_clients.loc[bank_clients.loc[:, 'family_status_id'] > 1, 
                 'family_status'] = 'не женат / не замужем'
print(bank_clients.groupby('family_status')['family_status_id'].value_counts())




family_status          family_status_id
гражданский брак       1                    4163
женат / замужем        0                   12344
не женат / не замужем  4                    4964
Name: family_status_id, dtype: int64


Настало время drop_duplicates!!!
Но дело в том, что если на этом этапе удалять дубликаты, есть шанс, что у нас появятся лишние дубликаты, т.к. часть таблицы я заполнил медианами. Проверим

In [36]:
x = bank_clients.duplicated().sum()
print('На данный момент дубликатов', x)

На данный момент дубликатов 21


### Вывод

Можно было объеденить и гражданский брак с разводом, но это не совсем верно
Можно сократить количество вариантов до 3х уже при вводе данных
Время покажет

После изменения семейного статуса количество дубликатов возросло дна 21. Однако, если данные были внесены с разными семейными статусами, из этого следует, что данные изначально были не идентичны. Поэтому я удалил дублирующиеся строки в самом начале работы

Проверю дубликаты после категоризации, ради интереса

### Лемматизация

Вероятнее всего лемматизация нужна для purpose, там очень много вариантов


In [37]:
print(bank_clients['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Ну и, собственно, сама лемматизация:

In [38]:
from pymystem3 import Mystem
m = Mystem()

text = bank_clients['purpose'].unique()
lemmas = m.lemmatize(' '.join(text))
from collections import Counter
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


### Вывод

Есть у меня чуечка, что не тем я занялся.
А если серьезно, то на "покупка" и "недвижимость" в топе. Но этот топ только по разношерстным упоминаниям. 
После сортировки списка можно прийти к меньшему количеству лемм

: 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})

1. 'покупка' (есть шанс что люди брали просто на покупку, после проверки, вероятно, этот вариант уберется)
2. 'недвижимость', 'жилье', 'строительство'
3. 'автомобиль'
4. 'образование'
5. 'операция' (снова, нужно проверить)
6. 'свадьба' (это просто зря)
7. 'приобретение' (та же история что и с "покупкой)
8. 'сыграть' (надеюсь, что не в казино, поэтому проверю и уберу)
9. 'проведение' (скорее всего это свадьба, но может быть и гадалка)
10. 'семья' (в проверку)
11. 'сдача' (в проверку)
12. 'ремонт' (в проверку) - ПОТРЕБИТЕЛЬСКИЙ

Оставил все существительные и "сыграть", т.к. было интересно, может, на казино люди брали деньги
Далее буду объеденять по категориям и применять

### Категоризация данных

In [39]:
text = bank_clients['purpose'].unique()
print(text)



['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [40]:
for query in text:
    if 'ремонт' in query:
        print(query)

ремонт жилью


ВЫВОД: ремонт жилью уходит в "потребительский кредит"

In [41]:
for query in text:
    if 'сдач' in query:
        print(query)

покупка жилья для сдачи


ВЫВОД: удаляем сдачу - она с жильем

In [42]:
for query in text:
    if 'семь' in query:
        print(query)


покупка жилья для семьи


ВЫВОД: семью удаляем, она с жильем

In [43]:
for query in text:
    if 'провед' in query:
        print(query)

на проведение свадьбы


ВЫВОД: проведение со свадьбой. удаляем

In [44]:
for query in text:
    if 'сыгр' in query:
        print(query)


сыграть свадьбу


ВЫВОД: сыграть тоже на свадьбу. удаляем

In [45]:
for query in text:
    if 'приобрет' in query:
        print(query)

приобретение автомобиля


ВЫВОД: к автомобилю отнесется. удаляем

In [46]:
for query in text:
    if 'свад' in query:
        print(query)

сыграть свадьбу
на проведение свадьбы
свадьба


ВЫВОД: вот оно зло. в потребительский свадьбу

In [47]:
for query in text:
    if 'операц' in query:
        print(query)

операции с жильем
операции с коммерческой недвижимостью
операции со своей недвижимостью
операции с недвижимостью


ВЫВОД: во всех вариантах есть и жилье и недвижимость - удаляем. все пойдет в недвижимость

In [48]:
for query in text:
    if 'образова' in query:
        print(query)

дополнительное образование
образование
заняться образованием
получение образования
получение дополнительного образования
получение высшего образования
профильное образование
высшее образование
заняться высшим образованием


ВЫВОД: все по учебе. а могло быть образование семьи. хотя и  то и то уходит в потребительский, или, давайте отдельно поместим. 
 интересно, много за учебу не отдают?

In [49]:
for query in text:
    if 'автом' in query:
        print(query)

приобретение автомобиля
на покупку подержанного автомобиля
на покупку своего автомобиля
автомобили
сделка с подержанным автомобилем
автомобиль
свой автомобиль
сделка с автомобилем
на покупку автомобиля


ВЫВОД: формулировок с авто гораздо больше. Запишем в ЛИЗИНГ, хотя это и не совсем верно

In [50]:
for query in text:
    if 'строит' in query:
        print(query)        

строительство собственной недвижимости
строительство недвижимости
строительство жилой недвижимости


ВЫВОД: во всех вариантах строительства есть жилье и недвижимость. удаляем

In [51]:
for query in text:
    if 'жил' in query:
        print(query)

покупка жилья
операции с жильем
покупка жилья для семьи
покупка жилой недвижимости
строительство жилой недвижимости
жилье
покупка своего жилья
покупка жилья для сдачи
ремонт жилью


ВЫВОД: с жильем все хорошо, кроме ремонта и операций. под операциями, вероятно имеются в виду сделки. 
А вот ремнт - совершенно другая категория. 
Поэтому при группировке в категории сначала нужно будет исправить ремонт, а после жилье

In [52]:
for query in text:
    if 'недвиж' in query:
        print(query)

покупка недвижимости
покупка коммерческой недвижимости
покупка жилой недвижимости
строительство собственной недвижимости
недвижимость
строительство недвижимости
операции с коммерческой недвижимостью
строительство жилой недвижимости
операции со своей недвижимостью
операции с недвижимостью


ВЫВОД: тут вск ОК, недвижимость она и в африке недвижимость

In [53]:
for query in text:
    if 'покуп' in query:
        print(query)

покупка жилья
покупка жилья для семьи
покупка недвижимости
покупка коммерческой недвижимости
покупка жилой недвижимости
на покупку подержанного автомобиля
на покупку своего автомобиля
покупка своего жилья
покупка жилья для сдачи
на покупку автомобиля


ВЫВОД: в покупке все недвижимость, жилье и автомобили. Все это было. Можно удалять
после нехитрых телодвижений у нас остались 4 категории (можно было и 3) и 5 ключевых слов
1. НЕДВИЖИМОСТЬ: 'недвижимость', 'жилье'
2. ЛИЗИНГ: 'автомобиль'
3. ОБРАЗОВАНИЕ: 'образование'
4. ПОТРЕБИТЕЛЬСКИЙ: 'ремонт', 'свадьба'

сейчас начинаем волшебство



In [54]:
def name_cat(x):
    if 'свад' in x:
        return 'потребительский'
    if 'ремо' in x:
        return 'потребительский'
    if 'образов' in x:
        return 'образование'
    if 'авто' in x:
        return 'лизинг'
    if 'недвиж' in x:
        return 'недвижимость'
    if 'жил' in x:
        return 'недвижимость'
    else:
        return 'хз'

bank_clients['b_d'] = bank_clients['purpose'].apply(name_cat)

print(bank_clients.groupby('b_d')['b_d'].count())

b_d
лизинг              4308
недвижимость       10207
образование         4014
потребительский     2942
Name: b_d, dtype: int64


In [55]:
x = bank_clients.duplicated().sum()
print('На данный момент дубликатов', x)

На данный момент дубликатов 21


### Вывод

То, что могло занять 20 минут, случилось за 220! 
А по таблице - около половины выдаваемых кредитов с банке на недвижимость. Фантастика. Есть шанс, что проценты по этому кедиту меньше

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [56]:
deti = bank_clients.groupby('children')['debt'].value_counts()
print(deti)

children  debt
0         0       13044
          1        1063
1         0        4411
          1         445
2         0        1926
          1         202
3         0         303
          1          27
4         0          37
          1           4
5         0           9
Name: debt, dtype: int64


Получилась таблица с интересущими нас данными, но как теперь добраться до нужных нам ячеек?
Пока оставим этот вопрос, потому как 3,4 и 5 детные люди не по своей численности составляют около 2х процентов. Объединим их в группу многоденых


In [57]:
def more_children(x):
    if x > 2:
        return 5
    return x
bank_clients['children'] = bank_clients['children'].apply(more_children)
deti = bank_clients.groupby('children')['children'].value_counts()
print(deti)

children  children
0         0           14107
1         1            4856
2         2            2128
5         5             380
Name: children, dtype: int64


In [58]:
deti = bank_clients.loc[:,('children','debt')]
print(deti)

       children  debt
0             1     0
1             1     0
2             0     0
3             5     0
4             0     0
...         ...   ...
21466         1     0
21467         0     0
21468         1     1
21469         5     1
21470         2     0

[21471 rows x 2 columns]


In [59]:
deti_grouped_sum = deti.groupby('children').sum()
deti_grouped_count = deti.groupby('children').count()
deti = pd.merge(deti_grouped_sum, deti_grouped_count, on=('children'))

посчитали общее кол-во людей и должников и объединили таблицы

In [60]:
deti['total'] = deti['debt_x'] / deti['debt_y']

In [61]:
deti.drop(['debt_x','debt_y'], axis='columns', inplace=True)
print(deti)

             total
children          
0         0.075353
1         0.091639
2         0.094925
5         0.081579


Посчитал проценты

In [62]:
def percent(x):
    fin = '{:.2%}'.format(x)
    return fin
deti['total'] = deti['total'].apply(percent)



In [63]:
print(deti)

          total
children       
0         7.54%
1         9.16%
2         9.49%
5         8.16%


### Вывод

Вероятность возврата кредита с появлением детей снижается. 
Люди с 2 детьми имеют большее количество задолженностей перед банком
Вопреки ожиданиям "многодетные" семьи возвращают кредиты охотнее

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

По накатанной: группируем и выводим таблицу

In [64]:
family = bank_clients.loc[:,('family_status','debt')]
family_sum = family.groupby('family_status').sum()
family_count = family.groupby('family_status').count()
family = pd.merge(family_sum, family_count, on=('family_status'))
family['total'] = family['debt_x'] / family['debt_y']

def percent(x):
    fin = '{:.2%}'.format(x)
    return fin
family['total'] = family['total'].apply(percent)
family.drop(['debt_x','debt_y'], axis='columns', inplace=True)

family


,total
family_status,
гражданский брак,9.32%
женат / замужем,7.54%
не женат / не замужем,8.50%


### Вывод

Процент долгов у семейных людей ниже. Зависимость есть.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [65]:

incom = bank_clients.loc[:,('total_income','debt')]

incom['total_income'] = incom['total_income'].astype('int')
#уменьшили цифры для читабельности и сгруппируем по зараоткам людей


def salary_count(incom):
    if incom < 50000:
        return 'a. до 50'
    if incom < 100000:
        return 'b. 50-100'
    if incom < 200000:
        return 'c. 100-200'
    return 'd. 200+'

incom['less_000'] = incom['total_income'].apply(salary_count)
#incom['less_000'] = incom['less_000'].astype('int')
incom.sort_values(by='less_000')


incom_sum = incom.groupby('less_000').sum()
incom_count = incom.groupby('less_000').count()
incom = pd.merge(incom_sum, incom_count, on=('less_000'))


incom['total'] = incom['debt_x'] / incom['debt_y']

def percent(x):
    fin = '{:.2%}'.format(x)
    return fin
incom['total'] = incom['total'].apply(percent)
#перевели в проценты


incom.drop(['debt_x','debt_y','total_income_x','total_income_y'], axis='columns', inplace=True)



incom


,total
less_000,
a. до 50,6.18%
b. 50-100,8.09%
c. 100-200,8.62%
d. 200+,7.07%


Этот вариант подсчета показаз, что люди с заработком 100-200к рублей имеют наибольшее количество задолженностей перед банком, 
Мирнимальные долги имеют люди, зарабатывающие до 50к
Следом за ними 200+ рублей

Далее пересчитаем по варианту,предложенному наставником

In [66]:
bank_clients['new'] = pd.qcut(bank_clients['total_income'],4)
bank_clients['new'].value_counts()

(195767.5, 2265604.0]    5368
(107654.5, 140116.0]     5368
(20666.999, 107654.5]    5368
(140116.0, 195767.5]     5367
Name: new, dtype: int64

Получилось 4 равных категории "до 107654", "до 140116", "до 195767", "более 195767.5"

In [67]:
incom_2 = bank_clients.loc[:,('total_income','debt')]

incom_2['total_income'] = incom_2['total_income'].astype('int')
def salary_count(incom):
    if incom < 107654:
        return 'a. 0 - 110'
    if incom < 140116:
        return 'b. 110-140'
    if incom < 195767:
        return 'c. 140-195'
    return 'd. 195+'

incom_2['up_to_000'] = incom_2['total_income'].apply(salary_count)

incom_new = incom_2.pivot_table(index=['up_to_000'], values=('debt'), aggfunc=('sum','count'))



incom_new['total'] = incom_new['sum'] / incom_new['count']

def percent(x):
    fin = '{:.2%}'.format(x)
    return fin
incom_new['total'] = incom_new['total'].apply(percent)

incom_new.drop(['sum','count'], axis='columns', inplace=True)

incom_new


,total
up_to_000,
a. 0 - 110,7.95%
b. 110-140,8.59%
c. 140-195,8.76%
d. 195+,7.13%


Мы получили практически те же данные, вторая и третья группы склонны отдавать свои долги реже, нежели 2 оставшиеся группы
За исключением измененных сумм заработков
Однако смело можно сделать вывод, что люди, зарабатывающие от 100 до 200к являются неплательщиками больше, нежели остальные

### Вывод

Есть вероятность неверного распределения категорий, однако по данным следует, что люди зарабатывающие менее 100.000 рублей имеют меньше неоплаченных кредитов, так же, как и люди, зарабатывающие больше 200.000

Самая "небезопасная" категория людей с заработками от 100 до 200 тысяч рублей
Так же в таблице были данные с людьми с нулевыми заработками, и пропущенным местом работы. Исходя из процентов невозврата можно предположить, что их заработки находятся в пределах от 200 до 400 тыс рублей. 
Я не силен в экономике России, но смею предположить что это самозанятые люди не трудоустроинные официально.


- Как разные цели кредита влияют на его возврат в срок?

Создадим сгруппированную таблицу по целям кредита

In [68]:
data_new = bank_clients.pivot_table(index=['b_d'], values=('debt'), aggfunc=('sum','count'))
data_new

,count,sum
b_d,,
лизинг,4308,403
недвижимость,10207,747
образование,4014,370
потребительский,2942,221


Добавим столбец с процентами

In [69]:
data_new['total'] = data_new['sum'] / data_new['count']
data_new

,count,sum,total
b_d,,,
лизинг,4308,403,0.093547
недвижимость,10207,747,0.073185
образование,4014,370,0.092177
потребительский,2942,221,0.075119


Переведем столбец Тотал в проценты и удалим ненужные столбцы

In [70]:
def percent(x):
    fin = '{:.2%}'.format(x)
    return fin
data_new['total'] = data_new['total'].apply(percent)

data_new.drop(['count','sum'], axis='columns', inplace=True)

data_new

,total
b_d,
лизинг,9.35%
недвижимость,7.32%
образование,9.22%
потребительский,7.51%


In [71]:
incom_new

,total
up_to_000,
a. 0 - 110,7.95%
b. 110-140,8.59%
c. 140-195,8.76%
d. 195+,7.13%


In [72]:
incom

,total
less_000,
a. до 50,6.18%
b. 50-100,8.09%
c. 100-200,8.62%
d. 200+,7.07%


In [73]:
family

,total
family_status,
гражданский брак,9.32%
женат / замужем,7.54%
не женат / не замужем,8.50%


### Вывод

Разница очевидна. Люди, берущие кредиты на авто и образование возвращают их реже, чем семейные люди, покупающие жилье и делающие свадьбы. Хотя это и странно. 

### Шаг 4. Общий вывод

Проведенное исследование показало большое количество пропущенных данных, которые являются важными при выдаче кредита
Из этого можно сделать вывод о возможности улучшить работу людей, заполнявших базу

Итоги исследования показали, что наличие 1 или 2 детей уменьшает вероятность возвращения кредита, однако многодетные люди охотнее возвращают, нежели люди с 1 и 2 детьми. Однако отсутствие детей при выдаче кредита являесть преимуществом

Что касается семейного положения, люди не в отношениях являютя самыми надежными, за ними следуют семейные люди. 
Самыми неблагонадежными являются люди в гражданском браке

По типу кредитования вероятность возврата снижается в следующем порядке: недвижимость, потребительские, образование, лизинг


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.